# Set up
- Install the packages
- Import dependencies

In [10]:
%pip install pandas numpy matplotlib seaborn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import pandas as pd
import httpx
import json
from tqdm import tqdm
import ast

# Multi-task Pipeline

Load the dataset

In [12]:
df = pd.read_csv('cleaned_reviews_noempty.csv')
df.head()

,store_name,rating,review,reviewer_name
0,49 SEATS,5,wowowow great vibes and food!! super eccentric...,Hannah Eva
1,49 SEATS,4,We had the classic pasta and fish n chips with...,S dssp
2,49 SEATS,5,Its an amazing restaurant with good vibes,Sanjith
3,49 SEATS,5,great atmosphere,Vivian L
4,49 SEATS,5,Great atmosphere and vibes!,Jayden


Prompt setup

In [13]:
# Read single-task prompts from .txt files using a helper function
def read_prompt(filename):
    # Input: filename (str) - path to the .txt file containing the prompt
    # Output: prompt (str) - the prompt text read from the file
    with open(filename, 'r', encoding='utf-8') as f:
        return f.read()

return_type_prompt = "After clarifying, return a dictionary with key are Ad, Irr, Rant, or Val, value : 1 for detected, 0 for not detected."
multitask_prompt = read_prompt('few_shot_prompt.txt') 

Classify function:
- Input: dataframe 
- Output: dataframe with predictions

multitask_model_name:
- for qwen-3-8b: "qwen/qwen3-8b:free"
- for gemma-3-12b: "google/gemma-3-12b-it:free"

In [ ]:
def classify_df(df, api_key):
    headers = {
        "Authorization": f"Bearer {api_key}",
        "X-Title": "Multitask Review Classification",
        "Content-Type": "application/json"
    }

    multitask_model_name = "google/gemma-3-12b-it:free" # plug the desired model name here

    reviews = df['review'].tolist()

    ad_preds = []
    irrelevant_preds = []
    rant_preds = []
    val_preds = []

    def classify_review(review, rating, store_name, reviewer_name):
        prompt = multitask_prompt + "\n" + return_type_prompt
        user_content = prompt.format(
            clean_text=review,
            rating=rating,
            store_name=store_name,
            reviewer_name=reviewer_name
        )
        data = {
            "model": multitask_model_name,
            "messages": [
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": user_content}
            ]
        }
        with httpx.Client() as client:
            resp = client.post(
                "https://openrouter.ai/api/v1/chat/completions",
                headers=headers,
                json=data,
                timeout=60
            )
            resp.raise_for_status()
            result = resp.json()
            return result["choices"][0]["message"]["content"]

    for idx in tqdm(range(len(reviews)), desc="Classifying reviews"):
        review = reviews[idx]
        rating = df.loc[idx, 'rating']
        store_name = df.loc[idx, 'store_name']
        reviewer_name = df.loc[idx, 'reviewer_name']
        pred = classify_review(review, rating, store_name, reviewer_name)
        # Extract the dictionary from the pred string
        start = pred.find('{')
        end = pred.rfind('}') + 1
        pred_dict = ast.literal_eval(pred[start:end])

        ad_preds.append(pred_dict.get('Ad', 'NA'))
        irrelevant_preds.append(pred_dict.get('Irr', 'NA'))
        rant_preds.append(pred_dict.get('Rant', 'NA'))
        val_preds.append(pred_dict.get('Val', 'NA'))

    multitask_df = pd.DataFrame({
        'review': reviews,
        'val_pred': val_preds,
        'ad_pred': ad_preds,
        'irrelevant_pred': irrelevant_preds,
        'rant_pred': rant_preds
    })
    multitask_df.to_csv('multitask_predictions.csv', index=False)
    return multitask_df

# To use:


Export function (from dataframe to csv)

In [21]:
def export_df_to_csv(df, filename):
    df.to_csv(filename, index=False)

API 1

In [15]:
api_1 = "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"

- Batch 1: 0 to 9 index

In [20]:
df_0_9 = df[:10]
df_0_9

,store_name,rating,review,reviewer_name
0,49 SEATS,5,wowowow great vibes and food!! super eccentric...,Hannah Eva
1,49 SEATS,4,We had the classic pasta and fish n chips with...,S dssp
2,49 SEATS,5,Its an amazing restaurant with good vibes,Sanjith
3,49 SEATS,5,great atmosphere,Vivian L
4,49 SEATS,5,Great atmosphere and vibes!,Jayden
5,49 SEATS,5,Wonderful food and service!,Andy Lim
6,49 SEATS,5,"The atmosphere and service are excellent, enjo...",郑苡萱
7,49 SEATS,2,"Resturant price, hawker center quality (the bl...",s z
8,49 SEATS,5,Always enjoy my meal there. Take note sometime...,AG Lee
9,49 SEATS,5,"Atmosphere here is great, food wise is also aw...",lee shaoxuan


In [22]:
df_0_9_pred = classify_df(df_0_9, api_1)

Classifying reviews: 100%|██████████| 10/10 [01:35<00:00,  9.59s/it]


In [23]:
df_0_9_pred

,review,val_pred,ad_pred,irrelevant_pred,rant_pred
0,wowowow great vibes and food!! super eccentric...,1,0,0,0
1,We had the classic pasta and fish n chips with...,1,0,0,0
2,Its an amazing restaurant with good vibes,1,0,0,0
3,great atmosphere,1,0,0,0
4,Great atmosphere and vibes!,1,0,0,0
5,Wonderful food and service!,1,0,0,0
6,"The atmosphere and service are excellent, enjo...",1,0,0,0
7,"Resturant price, hawker center quality (the bl...",1,0,0,0
8,Always enjoy my meal there. Take note sometime...,1,0,0,0
9,"Atmosphere here is great, food wise is also aw...",0,1,0,0


In [24]:
export_df_to_csv(df_0_9_pred, "df_0_9_pred.csv")

- Batch 2: 10 to 19 index


In [29]:
df_10_19 = df[10:20]
df_10_19 = df_10_19.reset_index(drop=True)
df_10_19

,store_name,rating,review,reviewer_name
0,49 SEATS,5,Always coming back for Tom Yum Pasta!!,Kai Ra
1,49 SEATS,5,Its my first time here and the aglio oglio is ...,Roxanna Song
2,49 SEATS,5,Great food and atmosphere.,Pearlyn Tang
3,Restoran Todak (Orang Asli),3,Super nice rustic place to have dinner. Great ...,Fauroni Sukhaimi
4,Restoran Todak (Orang Asli),2,"Poor attitude, ordinary food, arrogant persona...",Kheng poh Tan
5,Restoran Todak (Orang Asli),3,"The price is average, the taste is OK, it is n...",Zen Hoo
6,Restoran Todak (Orang Asli),5,"Nice food! So crowded, the seafood very fresh ...",Elyse J
7,Restoran Todak (Orang Asli),1,Not cross_mark,Ady Ernanda
8,49 SEATS,5,Great place to eat with friends,effie soon
9,49 SEATS,4,We loved the seafood carbonara pasta!,lilian chan


In [30]:
df_10_19_pred = classify_df(df_10_19, api_1)

Classifying reviews: 100%|██████████| 10/10 [01:46<00:00, 10.69s/it]


In [31]:
df_10_19_pred

,review,val_pred,ad_pred,irrelevant_pred,rant_pred
0,Always coming back for Tom Yum Pasta!!,1,0,0,0
1,Its my first time here and the aglio oglio is ...,1,0,0,0
2,Great food and atmosphere.,1,0,0,0
3,Super nice rustic place to have dinner. Great ...,1,0,0,0
4,"Poor attitude, ordinary food, arrogant persona...",0,0,0,1
5,"The price is average, the taste is OK, it is n...",1,0,0,0
6,"Nice food! So crowded, the seafood very fresh ...",1,0,0,0
7,Not cross_mark,0,0,0,1
8,Great place to eat with friends,1,0,0,0
9,We loved the seafood carbonara pasta!,1,0,0,0


In [32]:
export_df_to_csv(df_10_19_pred, "df_10_19_pred.csv")

- Batch 3: 20 to 29 index

In [34]:
df_20_29 = df[20:30]
df_20_29 = df_20_29.reset_index(drop=True)
df_20_29
df_20_29_pred = classify_df(df_20_29, api_1)
df_20_29_pred

Classifying reviews: 100%|██████████| 10/10 [00:59<00:00,  5.92s/it]


,review,val_pred,ad_pred,irrelevant_pred,rant_pred
0,chill space to unwind after a long week. tom ...,1,0,0,0
1,The service here is very good My food came ver...,1,0,0,0
2,very good and very nice emmanuel was a very go...,1,0,0,0
3,food very nice and service very good!,1,0,0,0
4,Have been eating here since close to a decade ...,1,0,0,0
5,"Great food, great location, great ambience!",1,0,0,0
6,Good Tom yum pasta. More on sour side. Nice ch...,1,0,0,0
7,Always love the tom yum seafood cream pasta!,1,0,0,0
8,Great service,1,0,0,0
9,Food is delicious! Staffs are friendly! Will b...,1,0,0,0


In [35]:
export_df_to_csv(df_20_29_pred, "df_20_29_pred.csv")

API 2: 

In [41]:
api_2 = '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'

- Batch 4: 30 to 39 index

In [42]:
df_30_39 = df[30:40]
df_30_39 = df_30_39.reset_index(drop=True)
df_30_39
df_30_39_pred = classify_df(df_30_39, api_2)
df_30_39_pred

Classifying reviews: 100%|██████████| 10/10 [00:57<00:00,  5.74s/it]


,review,val_pred,ad_pred,irrelevant_pred,rant_pred
0,Tom yam pasta spicy level 49 makes you want to...,1,0,0,0
1,The smoked duck pasta has always been my favou...,1,0,0,0
2,Nice vibes and great pasta!,1,0,0,0
3,地点和环境是真不错，可惜食物每次都有点失望disappointed_face,1,0,0,0
4,Delicious foods! My friends and I were having ...,1,0,0,0
5,We went for dinner. Busy as always. But still ...,1,0,0,0
6,Good food. Service depends on how crowded. No ...,1,0,0,0
7,超级烂服务，尤其华人服务员的态度都非常不友善！欠他们几百万酱紫！再来食物自己没介绍好menu...,0,0,0,1
8,I worked in a famous seafood restaurant in Sin...,1,0,0,0
9,This is my first try and the last food. The fo...,0,0,0,1


In [43]:
export_df_to_csv(df_30_39_pred, "df_30_39_pred.csv")

- Batch 5: 40 to 49 index

In [44]:
df_40_49 = df[40:50]
df_40_49 = df_40_49.reset_index(drop=True)
df_40_49
df_40_49_pred = classify_df(df_40_49, api_2)
df_40_49_pred

Classifying reviews: 100%|██████████| 10/10 [00:55<00:00,  5.56s/it]


,review,val_pred,ad_pred,irrelevant_pred,rant_pred
0,"Went for lunch today , ordered a crab and a re...",1,0,0,0
1,Come early before sunset for dinner if you wan...,1,0,0,0
2,Don't spend your money HERE ! ! Paying superio...,1,0,0,0
3,"Been many times, but my recent visit just 2 we...",1,0,0,0
4,Among the most beautiful things to see in Kual...,1,0,0,0
5,Yummy tom yum pasta thumbs_up_light_skin_tone,0,0,1,0
6,lovely place to dine in! the food is really de...,1,0,0,0
7,"Tom yum pasta is nice, highly recommend. The f...",1,0,0,0
8,food is good with alot of seats,1,0,0,0
9,good environment ，nice foodsmiling_face_with_h...,1,0,0,0


In [45]:
export_df_to_csv(df_40_49_pred, "df_40_49_pred.csv")

- Batch 6: 50 to 59 index

In [46]:
df_50_59 = df[50:60]
df_50_59 = df_50_59.reset_index(drop=True)
df_50_59
df_50_59_pred = classify_df(df_50_59, api_2)
df_50_59_pred

Classifying reviews: 100%|██████████| 10/10 [01:13<00:00,  7.34s/it]


,review,val_pred,ad_pred,irrelevant_pred,rant_pred
0,vibe of the place is cute!,1,0,0,0
1,Super slow in serving. Two hungry kids waited ...,1,0,0,0
2,It is great place for family gathering. Specia...,1,0,0,0
3,Nice choices of food,1,0,0,0
4,"comforting, tasty food for supper at affordabl...",0,1,0,0
5,"Pasta is tasty, and service is really good.",1,0,0,0
6,i ordered the duck agio olio and it was BITTER...,1,0,0,0
7,very welcoming staff and cosy ambience! famous...,1,0,0,0
8,49seats is amazing the food here is yummy and ...,1,0,0,0
9,Delicious food and friendly staff!,1,0,0,0


In [47]:
export_df_to_csv(df_50_59_pred, 'df_50_59_pred.csv')

API 3: 

In [49]:
api_3 = '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'

- Batch 7: 60 to 69 index

In [51]:
df_60_69 = df[60:70]
df_60_69 = df_60_69.reset_index(drop=True)
df_60_69
df_60_69_pred = classify_df(df_60_69, api_3)
df_60_69_pred

Classifying reviews: 100%|██████████| 10/10 [01:31<00:00,  9.13s/it]


,review,val_pred,ad_pred,irrelevant_pred,rant_pred
0,Food was delicious and fresh. Would come again,1,0,0,0
1,Very traditional place and reasonable price to...,1,0,0,0
2,"Nice place nice food, price a bit costly but s...",1,0,0,0
3,"food bad , waitress bad 1 5",0,0,0,1
4,Crab is nice. The rest of the dishes are average.,1,0,0,0
5,The food is ordinary and not very delicious. I...,1,0,0,0
6,Love the food and the environment at night for...,1,0,0,0
7,Was a little unsure about the reviews but was ...,1,0,0,0
8,I recently visited this restaurant. A spacious...,1,0,0,0
9,I was recommended this restaurant by a saloon ...,1,0,0,0


In [52]:
export_df_to_csv(df_60_69_pred, 'df_60_69_pred.csv')

- Batch 8: 70 to 79 index

In [53]:
df_70_79 = df[70:80]
df_70_79 = df_70_79.reset_index(drop=True)
df_70_79
df_70_79_pred = classify_df(df_70_79, api_3)
df_70_79_pred

Classifying reviews: 100%|██████████| 10/10 [01:26<00:00,  8.70s/it]


,review,val_pred,ad_pred,irrelevant_pred,rant_pred
0,"Reservation is a must, if not have to come ear...",1,0,0,0
1,"one bite of the fish, and i felt like im under...",1,0,0,0
2,Service is quite chaotic. u have to find yr ow...,1,0,0,0
3,The seafood was absolutely wonderful. For 16 p...,1,0,0,0
4,The road leading to the restaurant is kind of ...,1,0,0,0
5,thumbs_up The seafood here is super fresh and ...,1,0,0,0
6,Since 2002 loyal customer to this restaurant b...,1,0,0,0
7,We arrived at around 5pm for an early dinner s...,1,0,0,0
8,"The difference! Apart from the difference, it’...",0,0,0,1
9,The food taste has improved since the last tim...,1,0,0,0


In [54]:
export_df_to_csv(df_70_79_pred, 'df_70_79_pred.csv')

- Batch 9: 80 to 89 index

In [55]:
df_80_89 = df[80:90]
df_80_89 = df_80_89.reset_index(drop=True)
df_80_89
df_80_89_pred = classify_df(df_80_89, api_3)
df_80_89_pred

Classifying reviews: 100%|██████████| 10/10 [01:22<00:00,  8.25s/it]


,review,val_pred,ad_pred,irrelevant_pred,rant_pred
0,I came here after hearing so much about the pl...,1,0,0,0
1,"The food is fine, the food is not marked",0,0,1,0
2,The three sweet potato leaves have stones insi...,1,0,0,0
3,like the atmosphere,1,0,0,0
4,The food is delicious and the portion is large...,0,1,0,0
5,I’m here for the good food and vibes smiling_f...,1,0,0,0
6,The food is amazing and the staff is friendly.,1,0,0,0
7,The best tomyum pasta! Known them.since the kr...,1,0,0,0
8,"after many years, it never fails!",1,0,0,0
9,Love the tomyum pasta,1,0,0,0


In [56]:
export_df_to_csv(df_80_89_pred, 'df_80_89_pred.csv')

API 4:

In [58]:
api_4 = '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'

- Batch 10: 90 to 99 index

In [60]:
df_90_99 = df[90:100]
df_90_99 = df_90_99.reset_index(drop=True)
df_90_99
df_90_99_pred = classify_df(df_90_99, api_4)
df_90_99_pred

Classifying reviews:   0%|          | 0/10 [00:00<?, ?it/s]

Classifying reviews: 100%|██████████| 10/10 [01:20<00:00,  8.08s/it]


,review,val_pred,ad_pred,irrelevant_pred,rant_pred
0,Nice atmosphere and food.,1,0,0,0
1,how can you call your carbonara a carbonara wh...,1,0,0,0
2,Ordered the carbonara seafood. Sauce is creamy...,1,0,0,0
3,"Big portions of food, value for money.",1,0,0,0
4,"Cool vibes, interesting food choices!",1,0,0,0
5,Had a great dining experience at 49 Seats! The...,1,0,0,0
6,Cheap and good seafood. Value for money. Will...,1,0,0,0
7,Cheap but pay more for fresher array of seafoo...,1,0,0,0
8,Quite cheap to eat Seaafood especially crabs. ...,1,0,0,0
9,The place is amazing the staff is highly a...,1,0,0,0


In [61]:
export_df_to_csv(df_90_99_pred, 'df_90_99_pred.csv')

- batch 11: 100 to 109 index

In [63]:
df_100_109 = df[100:110]
df_100_109 = df_100_109.reset_index(drop=True)
df_100_109
df_100_109_pred = classify_df(df_100_109, api_4)
df_100_109_pred

Classifying reviews: 100%|██████████| 10/10 [00:48<00:00,  4.87s/it]


,review,val_pred,ad_pred,irrelevant_pred,rant_pred
0,"Revisited this place after 20+ years, still ju...",1,0,0,0
1,Went with some friends. Really enjoyed the dee...,1,0,0,0
2,The seafood was really impressive! The crab wa...,1,0,0,0
3,Lots of rubbish strewn over the car park and u...,1,0,0,0
4,"The Lobster is amazing, and the crab is a must...",1,0,0,0
5,Very packed. Have to wait 30 mins for table. G...,1,0,0,0
6,If you want LIVE seafood and at reasonable pri...,0,1,0,0
7,I think the food here is bad. Sometimes the ta...,0,0,0,1
8,Nice dinner at the seafront,1,0,0,0
9,Food is affordable and nice. Best of all no se...,1,0,0,0


In [64]:
export_df_to_csv(df_100_109_pred, 'df_100_109_pred.csv')

API 5:

In [68]:
api_5 = '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'

- batch 12: 110 to 119 index

In [70]:
df_110_119 = df[110:120]
df_110_119 = df_110_119.reset_index(drop=True)
df_110_119
df_110_119_pred = classify_df(df_110_119, api_5)
df_110_119_pred

Classifying reviews:   0%|          | 0/10 [00:00<?, ?it/s]

Classifying reviews: 100%|██████████| 10/10 [00:44<00:00,  4.49s/it]


,review,val_pred,ad_pred,irrelevant_pred,rant_pred
0,We were really hungry and found this place on ...,1,0,0,0
1,"Delay with meal , accidentally given to anothe...",1,0,0,0
2,the experience was excellent,1,0,0,0
3,What we had: Lunch Set (11.30am to 5pm) - Sta...,1,0,0,0
4,Found a delicious pasta near your home. The ki...,0,0,1,0
5,Food quite pricey & not very good. At the end ...,1,0,0,0
6,"Food is nice, service is fast but aircon needs...",1,0,0,0
7,A Hidden Gem for Fusion Pasta Lovers! 49 Seats...,1,0,0,0
8,food is good!,1,0,0,0
9,Good food and nice staff I recommend this rest...,1,0,0,0


In [71]:
export_df_to_csv(df_110_119_pred, 'df_110_119_pred.csv')

- batch 13: 120  to 129 index

In [73]:
df_120_129 = df[120:130]
df_120_129 = df_120_129.reset_index(drop=True)
df_120_129_pred = classify_df(df_120_129, api_5)
df_120_129_pred

Classifying reviews: 100%|██████████| 10/10 [00:41<00:00,  4.16s/it]


,review,val_pred,ad_pred,irrelevant_pred,rant_pred
0,Perfect for a day out with your kids. I would ...,1,0,0,0
1,Delicious food,1,0,0,0
2,Wide range of food options. Service provided b...,1,0,0,0
3,Cozy place. Good foods. Certainly will recomme...,1,0,0,0
4,Food is great. Ambiance is cozy Service is awe...,1,0,0,0
5,All Good! purple_heart,0,0,1,0
6,"The ambience was excellent, and the staff were...",1,0,0,0
7,Syahir very friendly and professional,1,0,0,0
8,Staff name syahir service fantastic,0,0,1,0
9,Syahir is very friendly and helpful.,1,0,0,0


In [74]:
export_df_to_csv(df_120_129_pred, 'df_120_129_pred.csv')

API 6:

In [81]:
api_6 = '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'

- batch 14: 130 to 139 index

In [83]:
df_130_139 = df[130:140]
df_130_139 = df_130_139.reset_index(drop=True)
df_130_139_pred = classify_df(df_130_139, api_6)
df_130_139_pred

Classifying reviews: 100%|██████████| 10/10 [01:02<00:00,  6.27s/it]


,review,val_pred,ad_pred,irrelevant_pred,rant_pred
0,Very good service Sha!,1,0,0,0
1,Beautiful food and nice atmosphere. Food is al...,1,0,0,0
2,"Nice ambiance, beautiful decoration. Friendly ...",1,0,0,0
3,Food very delicious. Environment very chillex,1,0,0,0
4,Air con a bit warm. But the rest is all good.,1,0,0,0
5,First time here - love the service and sambal ...,1,0,0,0
6,My go-to birthday spot!,1,0,0,0
7,"Great service provided, staff was very amicable.",1,0,0,0
8,Our first time dining in and we are very pleas...,1,0,0,0
9,Service and environment is good,1,0,0,0


In [84]:
export_df_to_csv(df_130_139_pred, 'df_130_139_pred.csv')

- batch 140 to 149 index

In [85]:
df_140_149 = df[140:150]
df_140_149 = df_140_149.reset_index(drop=True)
df_140_149_pred = classify_df(df_140_149, api_6)
df_140_149_pred

Classifying reviews: 100%|██████████| 10/10 [00:53<00:00,  5.38s/it]


,review,val_pred,ad_pred,irrelevant_pred,rant_pred
0,Great food and service and friendly staffs. Hi...,1,0,0,0
1,Good customer service,1,0,0,0
2,nice icecreamsmiling_face,0,0,1,0
3,everything’s aite but cherry on top was Belti’...,1,0,0,0
4,"Hi Team, this is was my 3rd time here overall…...",1,0,0,0
5,"Rethish, Arvin, Tin & Ning Ning has been reall...",1,0,0,0
6,"What's to say, but the utopian buffet experien...",1,0,0,0
7,"Avoid the set menus, go straight to a la carte...",1,0,0,0
8,Loved the food quality and selection here :) e...,1,0,0,0
9,庆祝朋友生日party_popperpartying_facebirthday_cake这里...,1,0,0,0


In [86]:
export_df_to_csv(df_140_149_pred, 'df_140_149_pred.csv')

API 7

In [ ]:
api_7 = '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'

- batch 150 to 159 index

In [94]:
df_150_159 = df[150:160]
df_150_159 = df_150_159.reset_index(drop=True)
df_150_159_pred = classify_df(df_150_159, api_7)
df_150_159_pred

Classifying reviews:   0%|          | 0/10 [00:00<?, ?it/s]

Classifying reviews: 100%|██████████| 10/10 [01:02<00:00,  6.27s/it]


,review,val_pred,ad_pred,irrelevant_pred,rant_pred
0,Service is excellent. I tried as much variety ...,1,0,0,0
1,"Airy, upscale hotel eatery with 7 open kitchen...",1,0,0,0
2,很豐富的自助餐，價格也很合理， 燒羊架、生蠔都是最是愛。 在訂位的時候告知是慶祝兒子生日，...,1,0,0,0
3,Just had the most horrible experience at Colon...,1,0,0,0
4,I have to go again to eat tiramishu,0,0,1,0
5,"It's delicious no matter what you eat, but it'...",0,0,1,0
6,Really dislike how the space is extremely crow...,1,0,0,0
7,Tasty food especially the Singapore Heritage C...,1,0,0,0
8,"The space especially the sushi section , the w...",1,0,0,0
9,"Food was delicious, staff were super friendly ...",1,0,0,0


In [95]:
export_df_to_csv(df_150_159_pred, 'df_150_159_pred.csv')

- batch 160 to 169 index

In [96]:
df_160_169 = df[160:170]
df_160_169 = df_160_169.reset_index(drop=True)
df_160_169_pred = classify_df(df_160_169, api_7)
df_160_169_pred

Classifying reviews: 100%|██████████| 10/10 [00:53<00:00,  5.39s/it]


,review,val_pred,ad_pred,irrelevant_pred,rant_pred
0,"Food was simply amazing and filling, location ...",1,0,0,0
1,Very friendly and helpful staff. I ordered fri...,1,0,0,0
2,Good food and vibes! Would come again :-),1,0,0,0
3,"Nice food, big portion",1,0,0,0
4,Amazing service and the food was great,1,0,0,0
5,"The food options are great, especially the bur...",1,0,0,0
6,good food in neighbourhood,1,0,0,0
7,Good Vibes Café really lives up to its name. C...,1,0,0,0
8,Great experience. Friendly staff.,1,0,0,0
9,"good vibes, friendly staff, picture worthy!",1,0,0,0


In [97]:
export_df_to_csv(df_160_169_pred, 'df_160_169_pred.csv')

- batch 170 to 179 index

In [98]:
df_170_179 = df[170:180]
df_170_179 = df_170_179.reset_index(drop=True)
df_170_179_pred = classify_df(df_170_179, api_7)
df_170_179_pred

Classifying reviews: 100%|██████████| 10/10 [00:58<00:00,  5.82s/it]


,review,val_pred,ad_pred,irrelevant_pred,rant_pred
0,staff is very friendly!,1,0,0,0
1,"the food here is delicious and verry filling ,...",1,0,0,0
2,halal certified and food portion is big,1,0,0,0
3,The food and environment is very nice and all ...,1,0,0,0
4,Halal and nice food and ice cream,1,0,0,0
5,Great place to hangout with your friends. Hala...,1,0,0,0
6,Great service and great food,1,0,0,0
7,Will come back againred_heart,0,0,1,0
8,"Good food and good services, definitely will c...",1,0,0,0
9,Huge portions and friendly staff 10 10,1,0,0,0


In [99]:
export_df_to_csv(df_170_179_pred, 'df_170_179_pred.csv')

API 8

In [106]:
api_8 = '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'

- batch 180 to 189 index

In [107]:
df_180_189 = df[180:190]
df_180_189 = df_180_189.reset_index(drop=True)
df_180_189_pred = classify_df(df_180_189, api_8)
df_180_189_pred
export_df_to_csv(df_180_189_pred, 'df_180_189_pred.csv')

Classifying reviews: 100%|██████████| 10/10 [01:01<00:00,  6.11s/it]


- batch 190 to 199 index

In [108]:
df_190_199 = df[190:200]
df_190_199 = df_190_199.reset_index(drop=True)
df_190_199_pred = classify_df(df_190_199, api_8)
df_190_199_pred
export_df_to_csv(df_190_199_pred, 'df_190_199_pred.csv')

Classifying reviews: 100%|██████████| 10/10 [01:05<00:00,  6.56s/it]


In [110]:
# Concatenate all prediction dataframes from df_0_9_pred to df_190_199_pred
dfs = [
    df_0_9_pred, df_10_19_pred, df_20_29_pred, df_30_39_pred, df_40_49_pred,
    df_50_59_pred, df_60_69_pred, df_70_79_pred, df_80_89_pred, df_90_99_pred,
    df_100_109_pred, df_110_119_pred, df_120_129_pred, df_130_139_pred,
    df_140_149_pred, df_150_159_pred, df_160_169_pred, df_170_179_pred,
    df_180_189_pred, df_190_199_pred
]

gemma_preds = pd.concat(dfs, ignore_index=True)
gemma_preds

,review,val_pred,ad_pred,irrelevant_pred,rant_pred
0,wowowow great vibes and food!! super eccentric...,1,0,0,0
1,We had the classic pasta and fish n chips with...,1,0,0,0
2,Its an amazing restaurant with good vibes,1,0,0,0
3,great atmosphere,1,0,0,0
4,Great atmosphere and vibes!,1,0,0,0
...,...,...,...,...,...
195,Wide variety of quality food on a monday lunch...,1,0,0,0
196,Good atmosphere and ambience with a fairly wid...,1,0,0,0
197,"First time trying this place, recommended by f...",1,0,0,0
198,Went for buffet lunch today and I must really ...,1,0,0,0


In [111]:
export_df_to_csv(gemma_preds, 'gemma_preds.csv')